<a href="https://colab.research.google.com/github/kodikarthik21/CS6910---Fundamentals-of-Deep-Learning/blob/main/Assignment01/CS6910_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

#Question 1 (2 Marks)
Download the fashion-MNIST dataset and plot 1 sample image for each class as shown in the grid below. Use "from keras.datasets import fashion_mnist" for getting the fashion mnist dataset.

﻿

In [ ]:
from keras.datasets import fashion_mnist
from matplotlib import pyplot
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
label = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
for i in range(10):
   wandb.init(project="cs6910-assignment01")
   wandb.run.name = "Q1_run_{}".format(i+1)
   for j in range(30):
     if y_train[j] == i:
       wandb.log({"examples": [wandb.Image(X_train[j], caption=label[i])]})
       break

# Question 2 (10 Marks)

Implement a feedforward neural network which takes images from the fashion-mnist data as input and outputs a probability distribution over the 10 classes.

Your code should be flexible so that it is easy to change the number of hidden layers and the number of neurons in each hidden layer.

In [ ]:
import numpy as np

def sigmoid(z):
  g = 1/(1+np.exp(-z))
  return g

def tanh(z):
  g = np.tanh(z)
  return g

def relu(z):
  return np.maximum(0,z)

In [ ]:
def softmax(x):
  return np.exp(x)/np.sum(np.exp(x), axis = 0)

In [ ]:
def initialize(initializer, size1, size2):
  if(initializer == "random"):
     W = np.random.randn(size1, size2)
     return W

  if(initializer == "Xavier"):
     W = np.random.randn(size1, size2) * np.sqrt(1/size2)
     return W

  print("Enter the name of initializer correctly")

In [ ]:
def linear_forward(H, W, b, activation):
  A = np.dot(W, H) + b 
  cache = (H, W, b)

  return A, cache

In [ ]:
def activation_forward(H_prev, W, b, activation):
  A, linear_cache = linear_forward(H_prev, W, b)
  activation_cache = A
  if activation == 'relu':
    H = relu(A)
  elif activation == 'sigmoid':
    H = sigmoid(A)
  elif activation =='tanh':
    H = tanh(A)
  elif activation == 'softmax':
    H = softmax(A)

  return H, activation_cache, linear_cache

In [ ]:
def initialize_Wb_matrix(layer_dims, initializer):
        
    np.random.seed(3)
    Wb_matrix = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        Wb_matrix['W' + str(l)] = initialize(initializer, layer_dims[l], layer_dims[l-1])
        Wb_matrix['b' + str(l)] = np.zeros((layer_dims[l], 1))
    
    return Wb_matrix

In [ ]:
def L_layer_model(X, Wb_matrix, activation):
    H = X
    L = len(Wb_matrix) // 2
    A_caches = []
    H_caches = [H]
    for l in range(1, L):
        H_prev = H 
        H, A_cache, linear_cache = activation_forward(H_prev, Wb_matrix['W{:d}'.format(l)], Wb_matrix['b{:d}'.format(l)], activation)
        A_caches.append(A_cache)
        H_caches.append(H)
    HL, A_cache = activation_forward(A, Wb_matrix['W%d' % L], Wb_matrix['b%d' % L], activation='softmax')
    A_caches.append(A_cache)
    return HL, H_caches, A_caches

In [ ]:
def forward_propagation(X, num_hidden, size_hidden, initializer, activation):

  layer_dims = [X.shape(0)]
  for l in range(1, num_hidden):
    layer_dims.append(size_hidden)
  layer_dims.append(10)
  
  Wb_matrix = initialize_Wb_matrix(layer_dims, initializer)
  y_hat, H_caches A_caches = L_layer_model(X, Wb_matrix, activation)

  return y_hat, H_caches, A_caches

# Question 3 (18 Marks)

Implement the backpropagation algorithm with support for the following optimisation functions 

- sgd
- momentum based gradient descent
- nesterov accelerated gradient descent
- rmsprop
- adam
- nadam

(12 marks for the backpropagation framework and 2 marks for each of the optimisation algorithms above)

We will check the code for implementation and ease of use (e.g., how easy it is to add a new optimisation algorithm such as Eve). Note that the code should be flexible enough to work with different batch sizes.

In [ ]:
def one_hot_encoding(l, L):
  e_y = []

  for i in range(L):
    if i == l:
      e_y.append(1)
    else:
      e_y.append(0)
  
  return np.array(e_y)

In [ ]:
def deriv_activ(z, activation):
  if(activation == "relu"):
    x[x<=0] = 0
    x[x>0] = 1
    return x

  elif(activation == "sigmoid"):
    g_deriv = sigmoid(z) * (1 - sigmoid(z))
    return g_deriv
  
  elif(activation == "tanh"):
    deriv = 1 - (tanh(z)) ** 2

In [ ]:
def back_propagation(H_caches, A_caches, Wb_matrix, y_hat, y, activation):
  e_y = one_hot_encoding(y, L)
  da = -(e_y - y_hat)
  grads = {}

  for k in reversed(range(1, L+1)):
    dW = np.dot(da, H_caches[k-1].T)
    db = da
    dh = np.dot(Wb_matrix['W{:d}'.format(k)].T, da)
    da = np.multiply(dh, deriv_activ(A_caches[k-1], activation))
    grads["dW" + str(k)] = dW
    grads["db" + str(k)] = db
  
  return dW_caches, db_caches

In [ ]:
def sgd(Wb_matrix, grads, eta):
  for l in range(L):
    Wb_matrix["W" + str(l+1)] = Wb_matrix["W" + str(l+1)] - eta * grads["dW" + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - eta * grads["db" + str(l+1)]
  return Wb_matrix

def momentum(Wb_matrix, grads, eta, gamma, update):
  update_new = {}
  for l in range(L):
    update_new["W" + str(l+1)] = (gamma * update["W" + str(l+1)]) + eta * grads["dW" + str(l+1)]
    Wb_matrix["W" + str(l+1)] = Wb_matrix["W" + str(l+1)] - update_new["W" + str(l+1)]

    update_new["b" + str(l+1)] = (gamma * update["b" + str(l+1)]) + eta * grads["db" + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - update_new["b" + str(l+1)]
  
  return Wb_matrix, update_new